# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import *
from tqdm.auto import tqdm # pip install tqdm
from MeasurementUnitaryGenerator import MeasurementUnitaryGenerator

Here we use $H_2$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [2]:
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(h2)
print('Number of fragments: {}\n'.format(len(qwc_list)))
for i, fragment in enumerate(qwc_list):
    print(f'Fragments {i} \n{fragment}\n')

Number of fragments: 5

Fragments 0 
-0.32760818967480915 [] +
0.049197645871367574 [X0 Y1 Y2 X3]

Fragments 1 
-0.049197645871367574 [X0 X1 Y2 Y3]

Fragments 2 
-0.049197645871367574 [Y0 Y1 X2 X3]

Fragments 3 
0.049197645871367574 [Y0 X1 X2 Y3]

Fragments 4 
0.13716572937099503 [Z0] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856082 [Z0 Z2] +
0.15542669077992838 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992838 [Z1 Z2] +
0.10622904490856082 [Z1 Z3] +
-0.13036292057109122 [Z2] +
0.1632676867356435 [Z2 Z3] +
-0.13036292057109122 [Z3]



By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_2$ by partitioning its terms into mutually commuting fragments. 

In [3]:
comm_groups = get_commuting_group(h2)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('\n')
print('The first commuting group')
print(comm_groups[1])
print('\n')
print('The second commuting group')
print(comm_groups[2])

Number of mutually commuting fragments: 2


The first commuting group
-0.32760818967480915 [] +
-0.049197645871367574 [X0 X1 Y2 Y3] +
0.049197645871367574 [X0 Y1 Y2 X3] +
0.049197645871367574 [Y0 X1 X2 Y3] +
-0.049197645871367574 [Y0 Y1 X2 X3] +
0.15660062488237958 [Z0 Z1] +
0.10622904490856082 [Z0 Z2] +
0.15542669077992838 [Z0 Z3] +
0.15542669077992838 [Z1 Z2] +
0.10622904490856082 [Z1 Z3] +
0.1632676867356435 [Z2 Z3]


The second commuting group
0.13716572937099503 [Z0] +
0.13716572937099503 [Z1] +
-0.13036292057109122 [Z2] +
-0.13036292057109122 [Z3]


To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [4]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999996+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [6]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-0.3276081896748089 [] +
0.15542669077992824 [X0] +
0.15660062488237952 [X0 X1] +
0.049197645871367546 [X0 X1 Z3] +
0.10622904490856076 [X0 X2] +
-0.049197645871367546 [X0 Z3] +
0.10622904490856076 [X1] +
0.15542669077992824 [X1 X2] +
-0.049197645871367546 [X1 X2 Z3] +
0.16326768673564335 [X2] +
0.049197645871367546 [X2 Z3]


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [7]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-0.3276081896748088 [] +
0.15542669077992818 [Z0] +
0.15660062488237944 [Z0 Z1] +
0.04919764587136753 [Z0 Z1 Z3] +
0.1062290449085607 [Z0 Z2] +
-0.04919764587136753 [Z0 Z3] +
0.1062290449085607 [Z1] +
0.15542669077992818 [Z1 Z2] +
-0.04919764587136753 [Z1 Z2 Z3] +
0.16326768673564326 [Z2] +
0.04919764587136753 [Z2 Z3]


## We created a class for generating measurement unitaries.
Please see MeasurementUnitaryGenerator.py
There is a demonstration of (experimental) SVD trunctation with H<sub>2</sub>O at the bottom of this notebook.

# H<sub>2</sub> (again - with the new framework)

In [3]:
mcalc = MeasurementUnitaryGenerator('h2')
print('\n')
print('First commuting group')
print(mcalc.comm_groups[1])
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First qwc unitary')
print(mcalc.uqwcs[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	4 orbitals
2) Optionally apply tapering
	1 orbitals
3) Get commuting groups
	2 commuting groups, a 1.50 x reduction from 3 terms
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First commuting group
-0.2764376754246701 [] +
0.17900057606140662 [X0]


First QWC group
-0.27643767542467007 [] +
0.1790005760614066 [Z0]


First Z group
-0.27643767542467007 [] +
0.1790005760614066 [Z0]


First qwc unitary
0.7071067811865475 [X0] +
0.7071067811865475 [Z0]


First z unitary
1.0 []


## H<sub>2</sub>O
Skip getting commuting groups as these steps are intractable. Skip straight ahead to getting QWC groups.

In [5]:
mcalc = MeasurementUnitaryGenerator('h2o', apply_uzs=True)
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	14 orbitals
2) Optionally apply tapering
	10 orbitals
More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups
	200 QWC groups, a 5.17 x reduction from 1035 terms
6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First QWC group
-46.35740160289526 [] +
0.0021618990497094402 [Y0 Z1 Z2 Z3 Z5 Z6 Y8] +
-0.0021618990497094402 [Y0 Z1 Z2 Y8] +
-0.0017952267657413302 [Y0 Z1 Z3 Z4 Z6 X7 Y8 X9] +
-0.004721276878475687 [Y0 Y8 X9] +
1.6462328281412955 [Z1] +
0.12333444961877364 [Z1 Z2] +
0.0037537905436206037 [Z1 Z2 Z3 Z5 Z6 X9] +
0.16866356664010346 [Z1 Z3] +
0.13693480693261512 [Z1 Z4] +
0.3378644911679983 [Z1 Z5] +
0.15105317022508946 [Z1 Z6] +
1.185904752943987 [Z2] +
0.14716615820292295 [Z2 Z3] +
0.002210090959302922 [Z2 Z3 Z4 Z6 X7 X9] +
0.13496957191310968 [Z2 Z4] +
0.30264028366319506 [Z2 Z5] +
0.1408322772493606 [Z2 Z6] +
1.294980147036568 [Z3] +
0.191189227748051 [Z3 Z4] +
0.3472114393571394 [Z3 Z5] +
0.11777797258841 [Z3 Z6] +
1.294980147036568 [Z4] +
0.3472114393571394 [Z4 Z5] +
0.13636442553512224 [Z4 Z6] +
2.730246966069517 [Z5] +
0.2819519797578531 [Z5 Z6] +
0.7861029297052462 [Z6]


First Z group
-46.35740160289523 [] +
0.002161899049709439 [Z0 Z1 Z2 Z3 Z5 Z6 Z8] +
-0.002161899049709439 

## H<sub>4</sub>

In [24]:
mcalc = MeasurementUnitaryGenerator('h4', apply_uzs=True)
print('\n')
print('First commuting group')
print(mcalc.comm_groups[1])
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First qwc unitary')
print(mcalc.uqwcs[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	8 orbitals
2) Optionally apply tapering
	4 orbitals
3) Get commuting groups
	5 commuting groups, a 9.20 x reduction from 46 terms
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First commuting group
-0.6362710050108025 [] +
-0.07052385292238096 [X0 Z1 Z2] +
0.07052385292238096 [X0 Z2 Z3] +
0.028555850825589 [Y0 X1 Y3] +
-0.005103767726276638 [Y0 Y1 X3] +
0.07114732950498256 [Y0 Z1 Y2 Z3] +
-0.07114732950498256 [Y0 Y2] +
0.07114732950498256 [Z0 X1 Z2 X3] +
0.07114732950498256 [Z0 Y1 Z2 Y3] +
0.005103767726276638 [Z0 Z1 X2] +
-0.028555850825589 [Z0 X2 Z3] +
-0.07811611442288481 [X1 Y2 Y3] +
0.07811611442288481 [Y1 Y2 X3] +
0.32806903436877016 [Z1 Z3]


First QWC group
-0.636271005010802 [] +
0.07052385292238092 [Z0] +
-0.07052385292238092 [Z0 X1] +
0.0711473295049825 [Z0 X1 Z2] +
-0.005103767726276633 [Z0 X1 Z3] +
-0.0711473295049825 [Z0 Z2] +
0.02855585082558898 [Z0 Z3] +
0.32806903436876994 [X1] +
0.005103767726276633 [X1 Z2] +
0.07811611442288477 [X1 Z2 Z3] +
-0.0711473295049825 [X1 Z3] +
-0.02855585082558898 [Z2] +
-0.07811611442288477 [Z2 Z3] +
0.0711473295049825 [Z3]


First Z group
-0.6362710050108019 [] +
0.0705238529223809 [Z0] +
-0.0705238529223809

## LiH

In [8]:
mcalc = MeasurementUnitaryGenerator('lih', apply_uzs=True)
print('\n')
print('First commuting group')
print(mcalc.comm_groups[1])
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First qwc unitary')
print(mcalc.uqwcs[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	12 orbitals
2) Optionally apply tapering
	8 orbitals
3) Get commuting groups
	41 commuting groups
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First commuting group
-3.947119127137718 [] +
-0.0009148940971918733 [X0 Y1 Y2 Z3 Z4 Z5 Z7] +
-5.845579096240596e-05 [X0 X2 X3] +
0.0009148940971918733 [Y0 Y1 X2 Z3 Z4 Z5 Z7] +
-5.845579096240596e-05 [Y0 Y2 X3] +
0.011733623912046789 [Z0 X1 Z2 X3] +
-0.0028574469714684443 [Z0 Y1 Y3 Z4 Z5 Z6] +
-0.00010502189355807878 [Z0 Y1 Z3 Z4 Z5 Y6 X7] +
0.09486647602250253 [Z0 Z1 Z3 Z4 Z5 Z7] +
0.05362141072261479 [Z0 Z2] +
0.0010456709574065426 [Z0 Z2 X3 Z4 Z5 Y6 Y7] +
0.08860943826798089 [Z0 Z2 Z4 Z5] +
0.09041974869932769 [Z0 Z2 Z4 Z5 Z6 Z7] +
-0.0015406700896984202 [X1 X3] +
-2.844458479947759e-05 [Y1 Z2 Y3 Z4 Z5 Z7] +
0.010540187409003523 [Y1 Z2 Y3 Z6] +
0.0027757462268932272 [Y1 Z2 Y3 Z7] +
0.00010502189355807878 [Y1 Z2 Z3 Y6 X7] +
0.09899076578197372 [Z1 Z2 Z3 Z4 Z5 Z7] +
-0.0010456709574065426 [X3 Y6 Y7] +
0.008434569756845402 [X4 X5] +
0.008434569756845402 [Y4 Y5] +
0.2707726623751814 [Z4 Z5] +
0.11409163501021498 [Z6 Z7]


First QWC group
-3.947119127137732 [] +
0.05362141072261472 [X

## NH<sub>3</sub>
Skip getting commuting groups as these steps are intractable. Skip straight ahead to getting QWC groups.

Warning: to save time you might want to skip applying the z-basis transformations with `apply_uzs=False`

In [19]:
mcalc = MeasurementUnitaryGenerator('nh3', apply_uzs=True)
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	16 orbitals
2) Optionally apply tapering
	14 orbitals
More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups
	959 QWC groups, a 3.76 x reduction from 3609 terms
6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First QWC group
-34.413644501497416 [] +
-0.00081303365904289 [Y0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 Z8 Z9 Z10 Y11 X12 X13] +
1.0772098183363803 [Z1] +
0.13600689568262758 [Z1 Z2] +
0.10606016163902365 [Z1 Z3] +
0.13600689568262755 [Z1 Z4] +
0.10606016163902378 [Z1 Z5] +
0.14836407229820797 [Z1 Z6] +
0.12295464996510772 [Z1 Z7] +
0.13150100122363995 [Z1 Z8] +
0.10855121194291724 [Z1 Z9] +
0.13610484417123336 [Z1 Z10] +
0.7684575395870031 [Z2] +
0.14032184811826043 [Z2 Z3] +
0.10748692518481216 [Z2 Z4] +
0.11843189949596164 [Z2 Z5] +
0.12437215825428041 [Z2 Z6] +
0.13177108541979066 [Z2 Z7] +
0.10788739359761279 [Z2 Z8] +
0.12231181049825243 [Z2 Z9] +
0.10428256107393341 [Z2 Z10] +
0.7684575395870034 [Z3] +
0.11843189949596164 [Z3 Z4] +
0.10748692518481216 [Z3 Z5] +
0.13177108541979066 [Z3 Z6] +
0.12437215825428041 [Z3 Z7] +
0.12231181049825243 [Z3 Z8] +
0.10788739359761279 [Z3 Z9] +
0.1375221381619795 [Z3 Z10] +
0.7684575395870028 [Z4] +
0.14032184811826073 [Z4 Z5] +
0.12437215825428051 [Z4 Z6] +

## N<sub>2</sub>
Skip getting commuting groups as these steps are intractable. Skip straight ahead to getting QWC groups.

Warning: to save time you might want to skip applying the z-basis transformations with `apply_uzs=False`

In [18]:
mcalc = MeasurementUnitaryGenerator('n2', apply_uzs=True)
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	20 orbitals
2) Optionally apply tapering
	16 orbitals
More than 8 orbitals: skipping steps 3), 4) and 5) and directly getting qwc groups
	680 QWC groups, a 4.34 x reduction from 2951 terms
6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First QWC group
-66.82850881574544 [] +
9.563301651472461 [Z0] +
-8.745925056048934e-05 [Z0 X1 Y7 Z8 Z9 Z10 Z11 Z12 Y13] +
0.04758098481143379 [Z0 X1 X14 Z15] +
0.18023887084369627 [Z0 Z2] +
-0.005824718860252173 [Z0 Z2 Z3 Z4 Z5 Z6 X14] +
0.003212498244432343 [Z0 Z2 Z4 Z6 Y7 Z8 Z10 Z12 Y13 Z15] +
-0.16818202264181578 [Z0 Z2 Z4 Z6 Z8 Z10 Z12 Z15] +
0.16494698146964537 [Z0 Z3] +
0.1574850800510944 [Z0 Z4] +
0.16839241530030746 [Z0 Z5] +
0.1650909592962415 [Z0 Z6] +
0.1681931653702476 [Z0 Z8] +
0.1658266762913248 [Z0 Z9] +
0.17749543250218083 [Z0 Z10] +
0.17403023288530975 [Z0 Z11] +
0.17749543250218075 [Z0 Z12] +
0.20707041559534994 [Z0 Z15] +
0.0011148794780630508 [X1 Z2 Z3 Z4 Z5 Z6 Y7 Z8 Z9 Z10 Z11 Z12 Y13 X14] +
-0.0032834895171798494 [X1 Z2 Z3 Z4 Z5 Z6 Z15] +
1.7411017750776425 [Z2] +
0.12590654018141012 [Z2 Z3] +
0.10809968817868348 [Z2 Z4] +
0.13333369645075033 [Z2 Z5] +
0.11882440493025873 [Z2 Z6] +
0.148444898668402 [Z2 Z8] +
0.12379026212730872 [Z2 Z9] +
0.14445795674212672 [

In [130]:
from functools import reduce

# Identity 
I = np.asarray([[1.,0.],[0.,1.]])
# Pauli X
X = np.asarray([[0.,1.],[1.,0.]])
# Pauli Y
Y = np.asarray([[0.,-1j],[1j,0.]])
# Pauli Z
Z = np.asarray([[1, 0], [0, -1]])

def operatorfromstring(pauli_string, max_orbitals):
    """
    Generate a many-body operator from a list of single-qubit 
    pauli matrices.
    ---------------------------------------------------------
    Input    pauli_string: str , (ex: [X,Z,X,Y,...])
    
    Output   pauli_op: np.array, shape = [2**N,2**N]
    """
    op_list = []
    eye_num = 0
    
    for k in range(len(pauli_string)):
        if (pauli_string[k][1] == 'X'):
            op_list.append(X)
        elif (pauli_string[k][1] == 'Y'):
            op_list.append(Y)
        elif (pauli_string[k][1] == 'Z'):
            op_list.append(Z)
        else:
            op_list.append(I)
            
    if not op_list:
            op_list.append(I)
            
    if len(op_list) < max_orbitals:
        eye_num = max_orbitals - len(op_list)
        for e in range(eye_num):
            op_list.append(I)         

    return reduce(np.kron,op_list)

def hamiltonianQ(pauli_list, interactions, max_orbitals):
    """
    Generate a many-body hamiltonian
    ---------------------------------------------------------
    Input    pauli_list: list of pauli strings
             interactions: list of interaction strengths
    
    Output   hamiltonian: np.array, shape = [2**N,2**N]
    """
    N = max_orbitals
    hamiltonian = np.zeros((1<<N,1<<N),dtype=complex)
    for i,pauli in tqdm(enumerate(pauli_list), total=len(pauli_list)):
        hamiltonian += interactions[i]*operatorfromstring(pauli, N)
    return hamiltonian

def eigensolve(hamiltonian):
    """
    Compute ground state energy and wavefunction
    """
    (eigenvalues,eigenstates) = np.linalg.eigh(hamiltonian)
    return eigenvalues[0],eigenstates[:,0]


pauli_list = []
interaction_list = []
max_orbitals = 0
#Generate pauli operations list and coefficient list(interactions)
for pauli, interaction in trunc_hamq.terms.items():
    pauli_list.append(pauli)
    interaction_list.append(interaction)

#Identify Number Max Oribitals    
for x in list(trunc_hamq.terms.keys()):
    _max = 0
    for y in x:
        if y[0] > _max:
            _max = y[0]
max_orbitals = _max + 1
print(max_orbitals)

h2_matrix = hamiltonianQ(pauli_list, interaction_list, 10)
print(h2_matrix.shape)
# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)

print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

9



(1024, 1024)

The eigenvalues in the effective Hamiltonian: 
 [-93.26860559 -93.26860559 -93.26754809 ...  -5.52744095  -5.52686937
  -5.52686937]


## Demonstration of SVD truncation with H<sub>2</sub>O
The truncation allows as to apply FC partitioning in a reasonable amount of time (if we are to store the result for later use). This was not possible without SVD truncation.

In [7]:
mcalc = MeasurementUnitaryGenerator('h2o', apply_uzs=True, svd_truncation_thresh=50)
print('\n')
print('First QWC group')
print(mcalc.qwc_groups[1])
print('\n')
print('First Z group')
print(mcalc.z_groups[1])
print('\n')
print('First z unitary')
print(mcalc.uzs[1])

1) Initialise the molecule object in a given basis and with a provided mapping
	14 orbitals
1.5) Applying (experimental) SVD truncation
	 Reduced number of terms from 1086 to 726
2) Optionally apply tapering
	8 orbitals
3) Get commuting groups
	25 commuting groups, a 18.20 x reduction from 455 terms
4) For each commuting group get the unitary for transforming to qwc



5) Apply unitaries to get the qwcs



6) Get unitaries for rotating everything to the measurable z-basis



7) Finally, apply latter unitaries to move all qwc groups to z-basis





First QWC group
-49.92467561227807 [] +
0.007534638979133925 [Z0 Z1 Z2 Z5] +
-0.0038778906914301637 [Z0 Z1 Z5 X6] +
0.0023979598531481024 [Z0 Z2 Z3] +
0.005620334803878947 [Z0 Z3 X6] +
-0.0023979598531481024 [Z1 Z2 Z3] +
0.014344949913924435 [Z1 Z2 Z3 Z4 X6] +
-0.0009814622557134092 [Z1 Z2 Z5] +
0.001872841184818052 [Z1 Z3 Z4] +
0.0065378272498456975 [Z1 Z3 X6] +
-0.019146086763794822 [Z1 Z5 X6] +
-0.002800230165301063 [Z2 Z3] +
-0.0019902586634875687 [Z2 Z4 Z5 X6] +
0.0027628364944506764 [Z2 Z5] +
0.00790494860459345 [Z2 X6] +
-0.004124245319122281 [Z3 X6] +
-0.0037931595150485224 [Z4 Z5] +
0.0010472411329209544 [Z5 X6] +
0.0006621069555075297 [X6] +
0.12871675695493404 [X6 Z7]


First Z group
-49.924675612278065 [] +
0.007534638979133923 [Z0 Z1 Z2 Z5] +
-0.003877890691430163 [Z0 Z1 Z5 Z6] +
0.002397959853148102 [Z0 Z2 Z3] +
0.005620334803878946 [Z0 Z3 Z6] +
-0.002397959853148102 [Z1 Z2 Z3] +
0.014344949913924434 [Z1 Z2 Z3 Z4 Z6] +
-0.000981462255713409 [Z1 Z2 Z5] +
0.0018728411848